# Chapter 16: Design for Physical Operations

## Example 16.1: Single_Stage_Limestone_Calciner.sce

In [ ]:
//Kunii D., Levenspiel O., 1991. Fluidization Engineering(II Edition). Butterworth-Heinemann, MA, pp 491

//Chapter-16, Example 1, Page 404
//Title: Single-Stage Limestone Calciner
//==========================================================================================================

clear
clc

//INPUT
T=1000;//Operating temperature of calciner in degree celcius
deltaHr=1795;//Heat of reaction in kJ/kg
M1=0.1;//Molecular weight of Calcium carbonate in kg/mol
M2=0.056;//Molecular weight of CaO in kg/mol
M3=0.044;//Molecular weight of Carbon dioxide  in kg/mol
M4=0.029;//Molecular weight of Air in kg/mol
M5=0.029;//Molecular weight of Combustion gas in kg/mol
Cp1=1.13;//Specific heat of Calcium carbonate in kJ/kg K
Cp2=0.88;//Specific heat of CaO in kJ/kg K
Cp3=1.13;//Specific heat of Carbon dioxide in kJ/kg K
Cp4=1.00;//Specific heat of Air in kJ/kg K
Cp5=1.13;//Specific heat of Calcium carbonate in kJ/kg K
Tf=20;//Temperature of feed in degree celcius
ma=15;//Air required per kg of fuel in kg
Hc=41800;//Net combustion heat of fuel in kJ/kg
Tpi=20;//Initial temperature of solids in degree C
Tgi=1000;//Initial temperature of gas in degree C

//CALCULATION
mc=1;//Based on 1 kg of Calcium carbonate
B=(1/(Hc-(ma+mc)*Cp5*(T-Tpi)))*[M3*Cp3*(T-Tf)+M2*Cp2*(T-Tf)+deltaHr]//Fuel consumption(kg fuel/kg calcium carbonate)
B1=B*M3/M2;//Fuel consumption(kg fuel/kg Cao)
H=Hc*B1;//Heat required for calcination
eta=deltaHr/(B*Hc);//Thermal efficiency

//OUTPUT
mprintf('
Fuel consumption:%f kg fuel/kg Cao',B1);
mprintf('
Heat requirement for calcination:%f kJ/kg Cao',H);
mprintf('
Thermal efficiency:%f percentage',eta*100);

//====================================END OF PROGRAM ======================================================

## Example 16.2: Multistage_Limestone_Calciner.sce

In [ ]:
//Kunii D., Levenspiel O., 1991. Fluidization Engineering(II Edition). Butterworth-Heinemann, MA, pp 491

//Chapter-16, Example 2, Page 405
//Title: Multistage Limestone Calciner
//==========================================================================================================

clear
clc

//INPUT
F=400;//Feed rate of Calcium carbonate in tons/day
T=1000;//Operating temperature of calciner in degree celcius
deltaHr=1795;//Heat of reaction in kJ/kg
M1=0.1;//Molecular weight of Calcium carbonate in kg/mol
M2=0.056;//Molecular weight of CaO in kg/mol
M3=0.044;//Molecular weight of Carbon dioxide  in kg/mol
M4=0.029;//Molecular weight of Air in kg/mol
M5=0.029;//Molecular weight of Combustion gas in kg/mol
Cp1=1.13;//Specific heat of Calcium carbonate in kJ/kg K
Cp2=0.88;//Specific heat of CaO in kJ/kg K
Cp3=1.13;//Specific heat of Carbon dioxide in kJ/kg K
Cp4=1.00;//Specific heat of Air in kJ/kg K
Cp5=1.17;//Specific heat of Combustion gas in kJ/kg K
Tf=20;//Temperature of feed in degree celcius
ma=15;//Air required per kg of fuel in kg
uo=0.8;//Superficial gas velocity in m/s
Hc=41800;//Net combustion heat of fuel in kJ/kg
Tpi=20;//Initial temperature of solids in degree C
Tgi=1000;//Initial temperature of gas in degree C
rhoa=1.293;//Density of air in kg/m^3
pi=3.14;

//CALCULATION
mc=1;//Based on 1 kg of Calcium carbonate
Bguess=2;//Guess value of B
function[fn]=solver_func(B)//Function defined for solving the system
    phi=((ma+mc)*Cp5*B+(M3*Cp3))/Cp1;
    T3=(Tpi+(phi+phi^2+phi^3)*Tgi)/(1+phi+phi^2+phi^3);
    phiplus=30.6*B
    Tr=(T+Tpi*phiplus)/(1+phiplus);
    fn=Hc*B+Cp3*(T3-Tpi)+ma*B*Cp4*(Tr-20)-(ma+mc)*Cp5*(T-Tpi)-M3*Cp3*(T-Tpi)-M2*Cp2*(T-Tpi)-deltaHr;
    //fn=(1/20800)*(2470-T3-13.34*(Tr-20));
endfunction
[B]=fsolve(Bguess,solver_func,1E-6);//Using inbuilt function fsolve for solving Eqn.(23) for tou
phi=((ma+mc)*Cp5*B+(M3*Cp3))/Cp1;
//Temperature of various stages
T1=(Tpi+(phi)*Tgi)/(1+phi);
T2=(Tpi+(phi+phi^2)*Tgi)/(1+phi+phi^2);
T3=(Tpi+(phi+phi^2+phi^3)*Tgi)/(1+phi+phi^2+phi^3);
phiplus=30.6*B
Tr=(T+Tpi*phiplus)/(1+phiplus);
eta=deltaHr/(B*Hc);//Thermal efficiency
H=B*Hc/M2;//Heat requirement
//For lower heat recovery section
Ql=(F*10^3/(24*3600))*B*ma/(rhoa*(273/(Tr+273)));//Volumetric flow rate of gas in the lower heat recovery section
dtl=sqrt(4/pi*Ql/uo);//Diameter of lower bed
//For calcination section
Qc=(F*10^3/(24*3600))*B*ma/(rhoa*(273/(T+273)));//Volumetric flow rate of gas in the calcination section
dtc=sqrt(4/pi*Qc/uo);//Diameter of calcination section
//For I stage
Q1=(F*10^3/(24*3600))*B*ma/(rhoa*(273/(T1+273)));//Volumetric flow rate of gas in the I stage
dt1=sqrt(4/pi*Q1/uo);//Diameter of I stage
//For II stage
Q2=(F*10^3/(24*3600))*B*ma/(rhoa*(273/(T2+273)));//Volumetric flow rate of gas in the II stage
dt2=sqrt(4/pi*Q2/uo);//Diameter of II stage
//For III stage
Q3=(F*10^3/(24*3600))*B*ma/(rhoa*(273/(T3+273)));//Volumetric flow rate of gas in the III stage
dt3=sqrt(4/pi*Q3/uo);//Diameter of III stage

//OUTPUT
printf('
Diameter of lower bed:%fm',dtl);
printf('
Diameter of calcination section:%fm',dtc);
printf('
Bed no.		1	2		3');
printf('
Diameter(m)%f	%f	%f',dt1,dt2,dt3);

//The value of diameter of each section is largely deviating from the values in the textbook. This is because the fuel consumption B have not been included in the energy balance equation. And the value of molecular weight is wrong by one decimal point.

//====================================END OF PROGRAM ======================================================

## Example 16.3: Multistage_Adsorber.sce

In [ ]:
//Kunii D., Levenspiel O., 1991. Fluidization Engineering(II Edition). Butterworth-Heinemann, MA, pp 491

//Chapter-16, Example 3, Page 413
//Title: Multistage Adsorber
//==========================================================================================================

clear
clc

//INPUT
T=20;//Temeprature in degree C
M=0.018;//Molecular weight of water in kg/mol
Q=10;//Flow rate of dry air in m^3/s
R=82.06E-6;//Universal gas constant
pi=0.0001;//Initial moisture content in atm
pj=0.01;//Final moisture content in atm

//CALCULATION
a=Q*(273+T)/273;//Term At*uo
b=a*M/(R*(T+273));//Term C*At*uo
//The value of slope can be found only by graphical mehtod. Hence it has been taken directly from the book(Page no.414,Fig.E3)
m=10.2;
Fo=b/m;//Flow rate of solids
Q3=(b/Fo)*(pj-pi);//Moisture content of leaving solids

//OUTPUT
printf('
Moisture content of leaving solids:%f kg H2O/kg dry solids',Q3);

//====================================END OF PROGRAM ======================================================

## Example 16.4: Dryer_Kinetics_and_Scale_up.sce

In [ ]:
//Kunii D., Levenspiel O., 1991. Fluidization Engineering(II Edition). Butterworth-Heinemann, MA, pp 491

//Chapter-16, Example 4, Page 422
//Title: Dryer Kinetics and Scale-up
//==========================================================================================================

clear
clc

//INPUT
Qfi=0.20;//Initial moisture fraction
Qfbar=0.04;//Average final moisture fraction
rhos=2000;//Density of solid in kg/m^3
Cps=0.84;//Specific heat of solids in kJ/kg K
Fo=7.6E-4;//Flow rate of solids in kg/m^3
Tsi=20;//Inital temperature of solids in degree C
rhog=1;//Density of gas in kg/m^3
Cpg=1;//Specific heat of gas in kJ/kg K
uo=0.3;//Superficial gas velocity in m/s
Tgi=200;//Initial temperature of gas in degee C
L=2370;//Enthalpy of liquid in kJ/kg
Cpl=4.2;//Specific heat of liquid in kJ/kg K
dt=0.1;//Diameter of reactor in m
Lm=0.1;//Length of fixed bed in m
ephsilonm=0.45;//Void fraction of fixed bed
pi=3.14;
Fo1=1;//Feed rate for commercial-scale reactor in kg/s

//CALCULATION
//(a)Bed temperature
Teguess=50;//Guess value of Te
function[fn]=solver_func(Te)//Function defined for solving the system
    fn=(pi/4)*dt^2*uo*rhog*Cpg*(Tgi-Te)-Fo*(Qfi-Qfbar)*[L+Cpl*(Te-Tsi)]-Fo*Cps*(Te-Tsi);
endfunction
[Te]=fsolve(Teguess,solver_func,1E-6);//Using inbuilt function fsolve for solving Eqn.(53) for Te

//(b)Drying time for a particle
xguess=2;//Guess value of x, ie term tou/tbar
function[fn]=solver_func1(x)//Function defined for solving the system
    fn=1-(Qfbar/Qfi)-(1-exp(-x))/x;
endfunction
[x]=fsolve(xguess,solver_func1,1E-6);//Using inbuilt function fsolve for solving Eqn.(61) for x
W=(pi/4)*dt^2*Lm*(1-ephsilonm)*rhos;//Weight of soilds in bed
tbar=W/Fo;//Mean residence time of solids from Eqn.(59)
tou=tbar*x;//Time for complete drying of a particle

//(c)Commercial-scale dryer
W1=Fo1*tbar;
Atguess=5;//Guess value of area
function[fn]=solver_func3(At)//Function defined for solving the system
    fn=At*uo*rhog*Cpg*(Tgi-Te)-Fo1*(Qfi-Qfbar)*[L+Cpl*(Te-Tsi)]-Fo1*Cps*(Te-Tsi);
endfunction
[At]=fsolve(Atguess,solver_func3,1E-6);//Using inbuilt function fsolve for solving Eqn.(53) for At
dt1=sqrt(4/pi*At);//Diameter of commercial-scale dryer
Q1=At*uo*rhog;//Flow rate necessary for the operation

//OUTPUT
printf('
Bed temperature:%f degree C',Te);
printf('
Time for complete drying of particle:%fs',tou);
printf('
Flow rate of gas necessary for Commercial-scale dryer:%fkg/s',Q1);

//====================================END OF PROGRAM ======================================================

## Example 16.5: Solvent_Recovery_from_Polymer_Particles.sce

In [ ]:
//Kunii D., Levenspiel O., 1991. Fluidization Engineering(II Edition). Butterworth-Heinemann, MA, pp 491

//Chapter-16, Example 5, Page 425
//Title: Solvent Recovery from Polymer Particles
//==========================================================================================================

clear
clc

//INPUT
rhos=1600;//Density of solid in kg/m^3
Cps=1.25;//Specific heat of solids in kJ/kg K
Fo=0.5;//Flow rate of solids in kg/s
Tsi=20;//Inital temperature of solids in degree C
Qwi=1;//Initial moisture fraction in water
Qwf=0.2;//Final moisture fraction in water
Qhi=1.1;//Initial moisture fraction in heptane
Qhf=0.1;//Final moisture fraction in heptane
Tgi=240;//Initial temperature of gas in degee C
Te=110;//Bed temperature in degree C
ephsilonm=0.45;//Void fraction of fixed bed
ephsilonf=0.75;//Void fraction of fluidized bed
uo=0.6;//Superficial gas velocity in m/s
di=0.08;//Diameter of tubes in m
li=0.2;//Pitch for square arrangement
hw=400;//Heat transfer coefficient in W/m^2 K
Tc=238;//Temperature at which steam condenses in degree C
//Specific heats in kJ/kg K
Cwl=4.18;//Water liquid
Cwv=1.92;//Water vapor
Chl=2.05;//Heptane liquid
Chv=1.67;//Heptane vapor
//Latent heat of vaporization in kJ/kg
Lw=2260;//Water
Lh=326;//Heptane
//Density of vapor in kg/m^3 at operating conditions
rhow=0.56;//Water
rhoh=3.1;//Heptane
Lf=1.5;//Length of fixed bed in m
t=140;//Half-life of heptane in s
L=1.5;//Length of tubes in heat exchanger
pi=3.14;

//CALCULATION
//(a) Dryer without Internals
xw=(Qwi-Qwf)/(Qhi-Qhf);//Water-heptane weight ratio
xv=((Qwi-Qwf)/18)/((Qhi-Qhf)/100);//Water-heptane volume ratio
T=(Qwi-Qwf)/18+(Qhi-Qhf)/100;//Total volume
rhogbar=((Qwi-Qwf)/18)/T*rhow+((Qhi-Qhf)/100)/T*rhoh;//Mean density of the vapor mixture
Cpgbar=(((Qwi-Qwf)/18)/T)*rhow*Cwv+(((Qhi-Qhf)/100)/T)*rhoh*Cwv;//Mean specific heat of vapor mixture
//Volumetric flow of recycle gas to the dryer in m^3/s from Eqn.(53)
x=(Cpgbar*(Tgi-Te))^-1*[Fo*(Qwi-Qwf)*[Lw+Cwl*(Te-Tsi)]+Fo*(Qhi-Qhf)*[Lh+Chl*(Te-Tsi)]+Fo*(Cps*(Te-Tsi))];
r=Fo*[(Qwi-Qwf)/rhow+(Qhi-Qhf)/rhoh};//Rate of formation of vapor in bed
uo1=uo*(x/(x+r));//Superficial velocity just above the distributor
At=x/uo1;//Cross-sectional area of bed
dt=sqrt(4/pi*At);//Diameter of bed
B=-log(Qwf/Qwi)/t;//Bed height from Eqn.(63)
tbar=((Qhi/Qhf)-1)/B;//Mean residence time of solids
W=Fo*tbar;//Weight of bed
Lm=W/(At*(1-ephsilonm)*rhos);//Static bed height
Lf=(Lm*(1-ephsilonm))/(1-ephsilonf);//Height of fluidized bed

//(b) Dryer with internal heaters
f=1/8;//Flow rate is 1/8th the flow rate of recirculation gas as in part (a)
x1=f*x;//Volumetric flow of recycle gas to the dryer in m^3/s from Eqn.(53)
uo2=uo*(x1/(x1+r));//Superficial velocity just above the distributor
Abed=x1/uo2;//Cross-sectional area of bed
q=[Fo*(Qwi-Qwf)*[Lw+Cwl*(Te-Tsi)]+Fo*(Qhi-Qhf)*[Lh+Chl*(Te-Tsi)]+Fo*(Cps*(Te-Tsi))]-Abed*uo2*Cpgbar*(Tgi-Te);//Heat to be added from energy balance of Eqn.(53)
Aw=q*10^3/(hw*(Tc-Te));//Total surface area of heat exchanger tubes
Lt=Aw/(pi*di);//Total length of tubes
Nt=Lt/L;//Total number of tubes
Atubes=Nt*(pi/4*di^2);//Total cross-sectional area of tubes
Atotal=Abed+Atubes;//Total cross-sectional area of tube filled dryer
d=sqrt(Atotal*pi/4);//Diameter of vessel
li=sqrt(Atotal/Nt);//Pitch for square array of tubes

//OUTPUT
printf('
			Bed diameter(m)	Recycle vapor flow(m^3/s)');
printf('
Without internal heater	%f	%f',dt,x);
printf('
With heating tubes	%f	%f',d,x1);

//====================================END OF PROGRAM ======================================================